In [1]:
import glob
import pandas as pd 

## 1) Download GAF files
http://current.geneontology.org/products/pages/downloads.html

## 2) Download associations
NCBI's gene2go file contains annotations of GO terms to Entrez GeneIDs for over 35 different species. We are interested in human which have the taxid 9606.

Follow the instructions in the [**background_genes_ncbi notebook**](https://github.com/tanghaibao/goatools/blob/main/notebooks/backround_genes_ncbi.ipynb) to download a set of background population genes from NCBI.



> 1. Query [NCBI Gene](https://www.ncbi.nlm.nih.gov/gene)
        "9606"[Taxonomy ID] AND alive[property]
> 2. Click "Send to:"
> 3. Select "File"
> 4. Select "Create File" button The default name of the tsv file is gene_result.txt
> 5. Convert NCBI Gene tsv file to a Python module    
    A goatools Python script will convert a NCBI Gene tsv file to a Python module:
>        `scripts/ncbi_gene_results_to_python.py gene_result.txt -o genes_ncbi_human.py`
> 6. Import NCBI data from Python module

___


## Load GAF files into the GafReader

In [4]:
%%time 
from goatools.anno.gaf_reader import GafReader

godata = {}
for gaf in glob.glob('goann/*.gaf'):
    name = gaf.split('/')[1].replace('.gaf','')
    godata[name] = {}
    godata[name]['Gaf'] = GafReader(gaf)

HMS:0:00:04.647915 146,309 annotations READ: goann/goa_human_isoform.gaf 
HMS:0:00:00.084926   2,136 annotations READ: goann/goa_human_complex.gaf 
HMS:0:00:01.572899  55,246 annotations READ: goann/goa_human_rna.gaf 
HMS:0:00:21.411174 609,183 annotations READ: goann/goa_human.gaf 
CPU times: user 23.1 s, sys: 2.81 s, total: 25.9 s
Wall time: 27.7 s


In [6]:
%%time 
for name in godata:
    godata[name]['GO Terms'] = [(x.GO_ID,x.DB_Name,x.NS) for x in godata[name]['Gaf'].get_associations()]
    print(name, len(godata[name]['GO Terms']))

GO_Terms_df = pd.DataFrame([(x,''.join(y),z) for name in godata for x,y,z in godata[name]['GO Terms']],columns=['GO_ID','DB_Name','Name_Space']).drop_duplicates('GO_ID').set_index('GO_ID')
len(GO_Terms_df)

goa_human_isoform 146309
goa_human_complex 2136
goa_human_rna 55246
goa_human 609183
CPU times: user 1.72 s, sys: 148 ms, total: 1.87 s
Wall time: 1.86 s


18854

## Read associations and map GO ids to gene names


In [7]:
# Read NCBI gene annotations 
from scripts.genes_ncbi_human import GENEID2NT # Already downloaded and converted to a python module ...

gnid2name = [(x.GeneID, x.Symbol) for x in GENEID2NT.values()]

gnid2name_df = pd.DataFrame(gnid2name,columns=['GeneID','Symbol']).set_index('GeneID')

In [8]:
len(gnid2name_df)

64013

In [9]:
from goatools.anno.genetogo_reader import Gene2GoReader

gene2go = Gene2GoReader('goann/gene2go', taxids=[9606]) # Already downloaded ...

HMS:0:00:10.343592 330,404 annotations, 20,688 genes, 18,642 GOs, 1 taxids READ: goann/gene2go 


In [10]:
# go2gnids = gene2go.get_id2gos(namespace='BP',go2geneids=True)
# go2gnids.update(gene2go.get_id2gos(namespace='CC',go2geneids=True))
# go2gnids.update(gene2go.get_id2gos(namespace='MF',go2geneids=True))

# go2gnnames = {}

# for key, value in go2gnids.items():
#     go2gnnames[key] = gnid2name_df.Symbol[value].to_list()

# len(go2gnnames)

In [11]:
gnid2gos = gene2go.get_id2gos(namespace='BP')
gnid2gos.update(gene2go.get_id2gos(namespace='CC'))
gnid2gos.update(gene2go.get_id2gos(namespace='MF'))

gnname2gos = {}

for key, value in gnid2gos.items():
    gnname2gos[gnid2name_df.Symbol[key]] = value
    
len(gnname2gos)

18501 IDs in loaded association branch, BP
19433 IDs in loaded association branch, CC
18194 IDs in loaded association branch, MF


20651

Subset to uniq GO ids

In [12]:
GOs = {go for val in gnname2gos.values() for go in val}
goann = GO_Terms_df.loc[GOs,:]

len(goann)

4886

## Write iPAGE annotations 

In [13]:
from ipage_down import *

In [14]:
mkdir -pv annotations/human_go_gs

In [15]:
write_page_index(gnname2gos,'annotations/human_go_gs/human_go_gs_index.txt.gz')

In [16]:
!zcat annotations/human_go_gs/human_go_gs_index.txt.gz | head -n 2

A2M	GO:0019966	GO:0048306	GO:0002020	GO:0019959	GO:0005515	GO:0005102	GO:0019838	GO:0043120	GO:0004866	GO:0019899	GO:0004867
NAT1	GO:0004060

gzip: stdout: Broken pipe


In [17]:
write_page_names(goann,'annotations/human_go_gs/human_go_gs_names.txt.gz')

In [18]:
!zcat annotations/human_go_gs/human_go_gs_names.txt.gz | head 

GO:0032051	Clathrin heavy chain	MF
GO:0046715	Sodium bicarbonate transporter-like protein 11	MF
GO:0071614	Cytochrome P450 2J2	MF
GO:0046965	Transforming acidic coiled-coil-containing protein 1	MF
GO:0003874	6-pyruvoyltetrahydropterin synthase	MF
GO:0004445	Inositol-polyphosphate 5-phosphatase	MF
GO:0050355	mRNA-capping enzyme	MF
GO:0033218	3-oxo-5-alpha-steroid 4-dehydrogenase 1	MF
GO:0004333	Fumarate hydratase, mitochondrial	MF
GO:0015420	Lysosomal cobalamin transporter ABCD4	MF

gzip: stdout: Broken pipe


## Methylation related pathways

In [19]:
set(GO_Terms_df.DB_Name[['methyl' in x for x in GO_Terms_df.DB_Name]].to_list())

{'(S)-3-amino-2-methylpropionate transaminase',
 '1,2-dihydroxy-3-keto-5-methylthiopentene dioxygenase',
 '2-C-methyl-D-erythritol 4-phosphate cytidylyltransferase-like protein',
 '2-methoxy-6-polyprenyl-1,4-benzoquinol methylase, mitochondrial',
 '3-hydroxy-3-methylglutaryl-coenzyme A reductase',
 '3-methyladenine DNA glycosidase',
 '4-trimethylaminobutyraldehyde dehydrogenase',
 '5-methylcytosine rRNA methyltransferase NSUN4',
 '6-O-methylguanine-DNA methyltransferase',
 "7-methylguanosine phosphate-specific 5'-nucleotidase",
 '7SK snRNA methylphosphate capping enzyme',
 'ATP synthase subunit C lysine N-methyltransferase',
 'Acetylserotonin O-methyltransferase',
 'Alpha N-terminal protein methyltransferase 1B',
 'Alpha-methylacyl-CoA racemase',
 'Aminomethyltransferase',
 'Arsenite methyltransferase',
 'Betaine--homocysteine S-methyltransferase 1',
 'Bifunctional arginine demethylase and lysyl-hydroxylase JMJD6',
 'Calmodulin-lysine N-methyltransferase',
 "Cap-specific mRNA (nucleosi

In [20]:
from iPAGE2 import ipage2

https://github.com/artemy-bakulin/iPAGE-2

In [21]:
!conda env export

name: down-go
channels:
  - defaults
  - anaconda
  - r
  - bioconda
  - conda-forge
dependencies:
  - _libgcc_mutex=0.1=main
  - _openmp_mutex=4.5=1_gnu
  - alabaster=0.7.12=py37_0
  - appdirs=1.4.4=py_0
  - atk-1.0=2.36.0=h28cd5cc_0
  - babel=2.9.1=pyhd3eb1b0_0
  - backcall=0.2.0=py_0
  - beautifulsoup4=4.9.3=pyha847dfd_0
  - bioservices=1.7.11=pyh3252c3a_0
  - blas=1.0=mkl
  - brotlipy=0.7.0=py37h27cfd23_1003
  - c-ares=1.17.1=h27cfd23_0
  - ca-certificates=2020.10.14=0
  - cairo=1.16.0=hf32fb01_1
  - certifi=2020.6.20=py37_0
  - cffi=1.14.5=py37h261ae71_0
  - chardet=4.0.0=py37h06a4308_1003
  - colorama=0.4.4=pyhd3eb1b0_0
  - colorlog=5.0.1=py37h06a4308_1
  - cryptography=3.4.7=py37hd23ed53_0
  - cycler=0.10.0=py37_0
  - decorator=4.4.2=py_0
  - docutils=0.17.1=py37h06a4308_1
  - easydev=0.10.1=pyh9f0ad1d_0
  - expat=2.4.1=h2531618_2
  - font-ttf-dejavu-sans-mono=2.37=h6964260_0
  - font-ttf-inconsolata=2.001=hcb22688_0
  - font-ttf-source-code-pro=2.030=h7457263_0
  - font-ttf-ubu